In [1]:
import pandas as pd
from pathlib import Path
import pymc as pm
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from estival.wrappers import pymc as epm
# from estival.wrappers import nevergrad as eng
# from estival.sampling import tools as esamp
# from estival.utils.parallel import map_parallel
import nevergrad as ng
from tbdynamics import model
from tbdynamics.utils import round_sigfig
from tbdynamics.inputs import load_params, matrix, conmat
from tbdynamics.constants import (
    age_strata,
    organ_strata,
    compartments,
    latent_compartments,
    infectious_compartments,
)
from tbdynamics.plotting import plot_model_vs_actual
import arviz as az
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Define variables

In [2]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / "data"
OUT_PATH = PROJECT_PATH / "output"
pd.options.plotting.backend = "plotly"

time_start = 1800.0
time_end = 2023.0
time_step = 0.1
matrix_homo = np.ones((6, 6))  # Homo mixing

In [3]:
fixed_parameters = load_params(PROJECT_PATH / "tbdynamics/params.yml")

## Define Model

### Base model

In [4]:
tb_model = model.build_model(
    compartments,
    latent_compartments,
    infectious_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    fixed_parameters,
    conmat,
)

### Params and calibration targets

In [5]:
params = {
    'smear_positive_death_rate': 0.339,
    'smear_negative_death_rate': 0.025,
    'smear_positive_self_recovery': 0.0192,
    'smear_negative_self_recovery': 0.073,
    'seed_num': 50.0, 
    'seed_duration': 1.0,
    'progression_multiplier':1.0, 
    'treatment_duration': 0.66, # 8 months
}


priors = [
    esp.UniformPrior("start_population_size", (5000000, 8000000)),
    esp.UniformPrior("contact_rate", (0.0001, 0.2)),  
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("progression_multiplier", (1.0, 5.0)),
    # esp.UniformPrior("treatment_duration", (0.5, 1.0)),
    esp.UniformPrior("seed_time", (1890.0, 1950.0)),
    esp.UniformPrior("seed_num", (1.0, 100.00)),
    esp.UniformPrior("seed_duration", (1.0, 5.0)),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
]

pop = pd.Series(
    {
        2009: 86025000,
        2019: 96484000,
    }
)

notifs = pd.Series(
    {
        2010: 99022,
        2011: 100518,
        2012: 103906,
        2013: 102196,
        2014: 102087,
        2015: 102676,
        2016: 102527,
        2017: 105733,
        2018: 102171,
        2019: 104505,
        2020: 101795,
    }
)

prop_smear_positive = pd.Series({2022: 0.83})
incidence = pd.Series({2022: 176})
percentage_latent= pd.Series({2022: 43.0})

targets = [
    est.NormalTarget("total_population", pop, stdev=100000.0),
    est.NormalTarget("notification", notifs, stdev=1000.0),
    est.NormalTarget("prop_smear_positive", prop_smear_positive, stdev=0.5),
    est.NormalTarget("percentage_latent", percentage_latent, stdev=5.0),
]

bcm = BayesianCompartmentalModel(tb_model, params, priors, targets)

### Running Optimization

In [8]:
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class)
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
   

In [9]:
mle_params


{'start_population_size': 5006906.430875297,
 'contact_rate': 0.00892500224291994,
 'rr_infection_latent': 0.4989783411290321,
 'rr_infection_recovered': 0.10093017285229183,
 'progression_multiplier': 1.0014651402500592,
 'seed_time': 1890.022301745532,
 'seed_num': 84.97100568845592,
 'seed_duration': 4.084341943373232,
 'smear_positive_death_rate': 0.44497144185404125,
 'smear_negative_death_rate': 0.03480226471845237,
 'smear_positive_self_recovery': 0.17767956533803472,
 'smear_negative_self_recovery': 0.20544975411141378}

In [12]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs

### Population output

In [11]:
plot_model_vs_actual(
    derived_df_0, pop, "total_population", "Population", "Modelled vs Data"
)

In [13]:
derived_df_0[[f"total_populationXage_{i}" for i in age_strata]].plot(
    title="Modelled populatation by age group", kind="area"
)

In [14]:
plot_model_vs_actual(
    derived_df_0, incidence, "incidence", "Incidence", "Modelled vs Data"
)

In [16]:
derived_df_0["prevalence_infectious"].plot()

In [17]:
derived_df_0[[f"prop_{compartment}" for compartment in compartments]].plot(kind="area")

In [18]:
derived_df_0[[f"total_populationXorgan_{i}" for i in organ_strata]].plot(
    title="Modelled populatation by organ status", kind="area"
)

In [19]:
plot_model_vs_actual(
    derived_df_0, notifs, "notification", "Notification", "Modelled vs Data"
)

In [20]:
plot_model_vs_actual(
    derived_df_0, percentage_latent, "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [ ]:
derived_df_0[[f"cdr_{organ_stratum}" for organ_stratum in organ_strata]].plot()